In [2]:
# Imports 
import numpy as np
import pandas as pd
import os
import pickle
import random
import matplotlib.pyplot as plt

# Image processing imports
from PIL import Image
from tensorflow.keras.utils import load_img
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model

# Progress bar
from tqdm.notebook import tqdm_notebook

# PCA and KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

2023-02-12 22:06:39.020591: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 22:06:40.405752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-12 22:06:40.405810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-12 22:06:40.405816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

> I'll eventually get around to dealing with these error messages.
>
> Apparently, these are *warnings* not errors.

> The following code is adapted from Gabe Flomo's [tutorial](https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34) on clustering images by similarity.


In [3]:
# Helper function for letterboxing an image into a square
def letterbox(img):
    width = img.size[0]
    height = img.size[1]
    size = max(width, height)
    new_img = Image.new(img.mode, (size, size), (255, 255, 255))
    
    if height > width:
        padding = int(0.5 * (height - width))
        new_img.paste(img, (padding, 0))
    else:
        padding = int(0.5 * (width - height))
        new_img.paste(img, (0, padding))
    return new_img

In [4]:
# Construct the model
model = VGG16()
# Remove the output layer
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
model

2023-02-12 22:06:41.472126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-12 22:06:41.472151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michael-XPS-8700): /proc/driver/nvidia/version does not exist
2023-02-12 22:06:41.472391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 22:06:41.646229: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2023-02-12 22:06:41.778809: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of fre

In [5]:
# Load and resize the images. The columns of the image_data list is vessel_ID
# followed by image_ID
vessel_ids = []
image_ids = []
images = []
feature_vectors = []
for filename in tqdm_notebook(os.listdir('../images/')):
    try:
        # Open the image, resize it, and convert it to a numpy array
        image = load_img('../images/' + filename)
        image = letterbox(image)
        image.thumbnail((224, 224))
        image = np.asarray(image)
        image = image.reshape(1, 224, 224, 3)
        
        # Convert the image to a feature vector
        image = preprocess_input(image)
        feature_vector = model.predict(image)
        
        # Get the vessel id and image id from the filename
        temp = filename.split('_')
        
        # Store the images, features, and ids in lists
        vessel_ids.append(temp[1])
        image_ids.append(temp[2][:-4])
        images.append(image)
        feature_vectors.append(feature_vector)
    except:
        # Keep track of the files we couldn't open.
        print('Could not open', filename)
        with open('bad_files.txt', 'a') as f:
            f.write(filename + '\n')

  0%|          | 0/10861 [00:00<?, ?it/s]

1/1 [==============================] - 0s 149ms/step
Could not open ._drawing_14769_8267.jpg
1/1 [==============================] - 0s 145ms/step
Could not open drawing_23361_25661.jpg
1/1 [==============================] - 0s 147ms/step
Could not open ._drawing_26731_33818.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_26671_33726.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_15893_10790.jpg
1/1 [==============================] - 0s 141ms/step
Could not open drawing_15881_10780.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_415_500.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_26672_33725.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_14784_8303.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_320_382.jpg
1/1 [==============================] - 0s 147ms/step
Could not open drawing_25

1/1 [==============================] - 0s 140ms/step
Could not open drawing_233_271.jpg
1/1 [==============================] - 0s 142ms/step
Could not open drawing_27640_36192.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_1099_1652.jpg
Could not open drawing_16834_33450.jpg
Could not open ._drawing_12186_3225.jpg
1/1 [==============================] - 0s 144ms/step
Could not open ._drawing_500_658.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_16053_11157.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_16260_11501.jpg
Could not open ._drawing_12250_3390.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_26984_34204.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_16040_11136.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_23352_25645.jpg
1/1 [==============================] - 0s 140ms/step


1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_26579_33832.jpg
1/1 [==============================] - 0s 142ms/step
Could not open drawing_27004_34184.jpg
1/1 [==============================] - 0s 160ms/step
Could not open ._drawing_1128_1732.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_27806_38857.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_11994_2832.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26889_34032.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_26888_34033.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_26663_33734.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_26881_34041.jpg
1/1 [==============================] - 0s 141ms/step
Could not open ._drawing_15262_9589.jpg
1/1 [==============================] - 0s 149ms/step
Could not ope

1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26759_33785.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26442_33490.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_481_626.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_16100_11228.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_27831_38891.jpg
1/1 [==============================] - 0s 149ms/step
Could not open drawing_26566_33683.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_17964_14649.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26643_33754.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_26635_33762.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_12213_3302.jpg
Could not open drawing_27654_36129.jpg
1/1 [=========================

...

1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_326_394.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_16134_11280.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_26883_34039.jpg
1/1 [==============================] - 0s 139ms/step
Could not open drawing_22785_24761.jpg
Could not open ._drawing_26675_33722.jpg
Could not open ._drawing_14691_8068.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_1139_1790.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_27342_35369.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_14733_8183.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26765_33779.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_1130_1742.jpg
Could not open ._drawing_16280_11536.jpg
1/1 [==============================] - 0s 138

1/1 [==============================] - 0s 139ms/step
Could not open drawing_27208_35121.jpg
1/1 [==============================] - 0s 143ms/step
Could not open drawing_388_462.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_27787_38840.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_16089_11208.jpg
1/1 [==============================] - 0s 139ms/step
Could not open drawing_26998_34190.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_27842_38899.jpg
1/1 [==============================] - 0s 139ms/step
Could not open drawing_15912_10809.jpg
1/1 [==============================] - 0s 139ms/step
Could not open drawing_15880_10779.jpg
Could not open drawing_27218_35090.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26744_33803.jpg
1/1 [==============================] - 0s 140ms/step
Could not open drawing_23352_25643.jpg
1/1 [==============================] 

1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_57_68.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_28698_40614.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_642_948.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_14774_8277.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_12267_3440.jpg
Could not open ._drawing_27929_37596.jpg
1/1 [==============================] - 0s 143ms/step
Could not open ._drawing_26533_33539.jpg
1/1 [==============================] - 0s 142ms/step
Could not open drawing_23352_25648.jpg
1/1 [==============================] - 0s 146ms/step
Could not open ._drawing_16057_11173.jpg
1/1 [==============================] - 0s 147ms/step
Could not open ._drawing_26578_33821.jpg
1/1 [==============================] - 0s 150ms/step
Could not open ._drawing_367_444.jpg
1/1 [==============================] 

1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_26553_33518.jpg
1/1 [==============================] - 0s 142ms/step
Could not open ._drawing_15175_9432.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_26738_33811.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_27838_38897.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_19799_16314.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_129_140.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_24410_27764.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_15149_9369.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_27011_34177.jpg
Could not open ._drawing_26031_33115.jpg
1/1 [==============================] - 0s 137ms/step
Could not open ._drawing_15856_10755.jpg
1/1 [========================

1/1 [==============================] - 0s 141ms/step
Could not open drawing_19800_16315.jpg
Could not open ._drawing_11488_1981.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_15916_10813.jpg
1/1 [==============================] - 0s 140ms/step
Could not open ._drawing_15968_10884.jpg
1/1 [==============================] - 0s 138ms/step
Could not open ._drawing_27745_38818.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_27648_36182.jpg
1/1 [==============================] - 0s 138ms/step
Could not open drawing_27007_34181.jpg
Could not open ._drawing_12290_3484.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_16122_11256.jpg
1/1 [==============================] - 0s 139ms/step
Could not open ._drawing_26703_33850.jpg
1/1 [==============================] - 0s 138ms/step


In [6]:
# Store the ids and images in a dataframe
df = pd.DataFrame(list(zip(vessel_ids, 
                           image_ids, 
                           images, 
                           feature_vectors)),
                  columns = ['vessel_ID',
                             'image_ID',
                             'image',
                             'feature_vectors'])
df.head()

,vessel_ID,image_ID,image,feature_vectors
0,18688,15819,"[[[[151.061 138.22101 131.32 ], [151.061 ...","[[0.37931985, 0.0, 0.15367365, 1.3902303, 0.0,..."
1,17496,13961,"[[[[151.061 138.22101 131.32 ], [151.061 ...","[[0.0, 0.3179601, 1.0356449, 0.0, 0.0, 0.0, 2...."
2,17334,13780,"[[[[151.061 138.22101 131.32 ], [151.061 ...","[[0.0, 0.68917066, 0.0, 2.6776593, 0.0, 0.0, 3..."
3,19762,18235,"[[[[151.061 138.22101 131.32 ], [151.061 ...","[[0.029503465, 0.4463769, 0.046591878, 0.70729..."
4,28755,40646,"[[[[151.061 138.22101 131.32 ], [151.061 ...","[[0.0, 0.113509804, 0.0, 0.0, 0.15061232, 0.0,..."


In [ ]:
df.to_pickle('image_data.pkl')

In [ ]:
feature_vectors = df['feature_vectors']

In [ ]:
# Load the pickle file
df = pd.read_pickle('image_data.pkl')

In [ ]:
# Get the feature vectors. I'm not sure why these steps work. I need to
# study this more.
fv = df['feature_vectors']
fv = np.array(list(fv))
fv = fv.reshape(-1, 4096)

In [ ]:
# Use PCA to compress the feature vectors
pca = PCA(n_components = 0.95)
pca.fit(fv)
fv = pca.transform(fv)

In [ ]:
# Get the shape of the transformed data
fv.shape

> Observe that we were able to reduce the feature vectors to a length of 688 as opposed to 4096 while keeping the explained variance at 95%. Not bad!

In [ ]:
# Save the compressed data to a pickle file
df['feature_vectors'] = list(fv)
#df.to_pickle('compressed_image_data.pkl')

> Compressing the feature vector only saved about 0.2 gigs of space. Most of the pickle file is due to the images. Since we can load the images at any time, it's probably not worth saving the images in the pickle file, and loading them as necessary using the saved ids.
>
> The following code is messy and uncommented. Needs to be cleaned up.

In [ ]:
# Find the optimal number of clusters
inertias = []
silhouette_scores = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(fv)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(fv, kmeans.labels_))

In [ ]:
# Graph the inertias
plt.figure(figsize = (13, 8))
plt.plot(range(2, 20), inertias, marker = '.')

In [ ]:
# Graph the silhouette scores
plt.figure(figsize = (13, 8))
plt.plot(range(2, 20), silhouette_scores, marker = '.')

> We will try two models. One using three clusters, and one using four clusters.

In [ ]:
# Build the three cluster model
kmeans3 = KMeans(n_clusters = 3)
kmeans3.fit(fv)

In [ ]:
# Build the four cluster model
kmeans4 = KMeans(n_clusters = 4)
kmeans4.fit(fv)

In [ ]:
# Append the labels for the three cluster model to the data frame
df3 = df.copy()
df3['labels'] = kmeans3.labels_
df3.head()

In [ ]:
# Append the labels for the four cluster model to the data frame
df4 = df.copy()
df4['labels']= kmeans4.labels_
df4.head()

In [ ]:
# Look at the counts of the clusters
display(df3['labels'].value_counts())
display(df4['labels'].value_counts())

In [ ]:
plt.imshow(df['image'][50].reshape(224, 224, 3))

In [ ]:
# Save the clusters
df3[['vessel_ID', 'image_ID', 'labels']].to_csv('three_cluster_model_labels.csv')
df4[['vessel_ID', 'image_ID', 'labels']].to_csv('four_cluster_model_labels.csv')
#df[['vessel_ID', 'image_ID', 'labels']].to_csv('cluster_labels_with_ids.csv')